# Welcome to TinyML part 2: Data preparation, modelling, training, and evaluation

 This is an interactive Jupyter notebook where you can run the Python code that we have prepared interactively.
 Below you will find cells that are either code or description.  


In [ ]:
import sys
print(sys.version)

### Import modules

Import libraries that we need to prepare data, work with a model, train and evaluate the results.

In [ ]:
import os
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten

from utils.calc_mem import calc_mem
from preprocess import preprocess, WINDOW_SIZE, SPECTRUM_SIZE, SPECTRUM_MEAN, SPECTRUM_STD, SAMPLE_RATE, FRAME_SIZE, FRAME_STRIDE
from utils.export_tflite import write_model_h_file, write_model_c_file
from utils.plots import plot_dataset, plot_learning_curves, plot_convolution_filters, plot_first_positive_window, plot_predictions_vs_labels

# Minimize TensorFlow logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

### Load and preprocess data

This code loads and preprocesses the data and label files.

**Training data format**

Training data should be placed in the `Data` folder. Each recording should provide two files:

* A .wav file with 16 bit mono audio in 16 kHz. Its file name should be `audio_<id>.wav`, where `<id>` can be any unique string that identifies the recording.
* A .txt file with labels indicating the starting and ending times of each bell sound in the recording. Each row contains a starting time in seconds, the ending time in seconds and a label name, separated by tab characters. This conforms to Audacity's label file format. Its file name should be `label_<id>.txt`, where `<id>` is the recording identifier.

**Signal processing**

The preprocessing of a single recording takes place in the `preprocess` function in `preprocess.py`. After loading the .wav file into a numpy array, the audio is split into frames of size `FRAME_SIZE` (currently 256 audio samples corresponding to 16 ms), and the following signal processing is performed on each frame:

* Subtract the frame average to remove any constant bias in the audio frame.
* Apply a Hamming window - common practice before FFT to avoid spectral leakage.
* Perform a real FFT (Fast Fourier Transform), which produces a spectral frame with 129 frequency bins. Frequency bin `i` contains the magnitude at frequency `i * SAMPLE_RATE / FRAME_SIZE`, so for example, bin 40 contains the magnitude at frequency 40 * 16000 Hz / 256 = 2500 Hz.
* Remove bins above 3000 Hz (bin 48), since they contain little relevant information and we want to keep input data small.
* Apply log(1 + x) to compress high magnitude values and obtain an even spread of values.
* Normalize the data to ensure it has approximately mean 0 and standard deviation 1, which helps neural network training.

**Sliding windows**

The sequence of all spectral frames constitute the complete spectrogram for the audio file. However, the classifier that we're about to train can neither work on a single spectral frame, because 16 ms of sound is too little to make classifications on, nor can it work on the entire file, because it's too big. Therefore, we use a sliding window over the spectrogram to cut out small spectrograms of length `WINDOW_SIZE` (currently 32 frames, corresponding to 256 ms). This window slides over the full spectrogram with a stride of `WINDOW_STRIDE` (currently 8 frames, corresponding to 64 ms). In other words, the windows are overlapping. Each such window will constitute an *input matrix*, or *feature matrix*, of size 32x48. Our classifier will take one such window and classify whether it contains a bell sound or not. With over 60 minutes of audio data, our entire data set contains more than 30,000 such windows.

**Labelling windows from time intervals**

Each window must have a label - `0` or `1` - to indicate to the training algorithm whether the window contains the bell sound or not. However, our label files contain time intervals in seconds, for example:

```
25.29	26.29	Ding
70.578	71.578	Ding
115.866	116.866	Ding
...
```

The window labeling algoritm at the end of the `preprocess` function assigns the label `1` if the window is completely within a time interval in the label file. For example:

* The window with index 395, starting at 25.28 s and ending at 25.536 s, is assigned label `0`, because it's not completely within the interval 25.29 - 26.29.
* The window with index 396, starting at 25.334 s and ending at 25.6 s, is assigned label `1`, because it's completely within the interval.

**Return shapes**

The `preprocess` function returns one numpy array `x` of shape `(n, 32, 48)`, containing all *n* windows (aka small spectrograms aka feature matrices) and one numpy array `y` with *n* labels for the recording. Since we have multiple recording files, these individual `x` and `y` for each file are gathered in a list and finally concatenated into one big `x` of `(N, 32, 48)` and `y` of shape `(N)`.

In [ ]:
# Load and preprocess data
data_dir = '../Data/'
x_files = []
y_files = []
for c_file in os.listdir('../Data/'):
    if c_file.startswith('audio_') and c_file.endswith('.wav'):
        recording_id = c_file[6:-4]
        label_file = 'labels_' + recording_id + '.txt'
        print('Preprocessing ' + c_file + ' and ' + label_file)
        x_file, y_file = preprocess(data_dir + c_file, data_dir + label_file)
        x_files.append(x_file)
        y_files.append(y_file)

# Concatenate files into feature and label arrays
x = np.concatenate(x_files)  # Shape: (number of windows, WINDOW_SIZE, SPECTRUM_SIZE) = (number of windows, 32, 48)
y = np.concatenate(y_files)  # Shape: (number of windows, 1)

# Plot the spectrogram of the entire dataset with labels underneath
plot_dataset(x, y, block=True)

### Split into training, validation and test sets and determine class weights

**Splitting data**

A common beginner mistake is to use all data for training. This is bad for two reasons:

* We have no means of controlling overfitting. *Overfitting* is when the model is trained "too well" on the data set, so it adapts to noise and outliers in the data set. It's typically caused by training a too big model on too little data. The consequence is that the model works extremely well on the training data, but when it's tested on new unheard or unseen data, it performs very badly. Overfitting is a common problem in machine learning scenarios, especially with smaller data sets.
* We have no way of evaluating the model's accuracy. Computing the accuracy on the same data that the model was trained on will give a very optimistic result.

The solution to split the entire data set into three subsets:

* **Training set**:  Used for training the model, i.e. optimizing the model parameters.
* **Validation set**: Used for *early stopping*, which means stopping the training when no improvement can be seen on the validation set. This is a common method to prevent overfitting.
* **Test set**: Used for computing a reliable accuracy.

**Class weights**

Another common beginner mistake is to have an unbalanced data set, which means to not have approximately the same number of examples for all classes, and not doing anything about it. This is clearly the case in our project, where the bell sounds only 1 out of 45 seconds. In fact, we have we have approximately 51 times more `0` examples (windows with no bell sound) than `1` examples (windows with bell sound). The problem with unbalanced data is that the training will favorize the overrepresented class, since it then appears more accurate when evaluated on the data set. Even a classifer that always outputs `0` regardless input would appear quite accurate with ~98% accuracy on our data set.

The best solution to this problem is to complement with more data from the underrepresented class (in our case, more bell sound data) to ensure that the data set is balanced. If that's not possible, one can compensate by providing *class weights* to TensorFlow, telling it to weigh the examples labeled with a certain class higher. Since we have about 51 times more `0` exaples than `1` examples, we give class `1` a weight that is 51 times higher than that for class `0`.

In [ ]:
# Split into training, validation and test sets
x_train, x_val, x_test = np.split(x, [int(.6 * len(x)), int(.8 * len(x))])
y_train, y_val, y_test = np.split(y, [int(.6 * len(y)), int(.8 * len(y))])

# Determine class weights
num_positives = np.sum(y)
num_negatives = len(y) - num_positives
ratio = num_negatives / num_positives
class_weights = {0: 1 / np.sqrt(ratio), 1: np.sqrt(ratio)}  # Divide by sqrt(ratio) to make losses comparable
print('Class weights:', class_weights)

### Build and compile the model

We choose a model consisting of four layers:

* A `Conv1D` layer with 8 filters of size 3 to detect raw patterns over 3 frames (48 ms).
* Another `Conv1D` layer with 8 filters of size 3 to detect combined patterns over 5 frames (80 ms).
* A `Flatten` layer to turn convolution output to a vector; required for Dense.
* A `Dense` layer with a single neuron to summarize into prediction output.

With `model.compile()`, the model is combined with:

* A *loss function* to be minimized, in this case `binary_crossentropy`, which is the default choice for binary classifiers.
* An *optimization algorithm* for minimizing the loss, in this case `adam`, which is typically the default choice.


In [ ]:
# Build and compile model
print('Building model...')
model = Sequential()
model.add(Conv1D(8, 3, activation='relu', input_shape=(WINDOW_SIZE, SPECTRUM_SIZE)))  # Output shape (30, 8)
model.add(Conv1D(8, 3, activation='relu'))  # Output shape (28, 8)
model.add(Flatten())  # Output shape (224,)
model.add(Dense(1, activation='sigmoid'))  # Output shape (1)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

### Train the model

Training runs in iterations called *epochs*. For each epoch, the following happens:

* The optimizer runs over the entire training data set one batch (128 windows) at a time. For each batch:
    * The loss function is computed on the batch.
    * The gradient of the loss function is computed using backpropagation.
    * The parameters are updated using the gradient.
* The callbacks are called:
    * `early_stopping`: If the validation loss has not improved for 16 epochs, training stops.
    * `model_checkpoint`: If the validation loss is the lowest so far, the model and parameters are saved to a file.

The learning curve plot shows the value of the loss function over the epochs for both the training set and the validation set. The learning curve provides valuable information about how well the training went. In particular:

* The training loss should decrease quite smoothly. If not, training is unstable and hyper-parameters like learning rate should be tuned, and the data should be (better) shuffled.
* The validation loss should also decrease, but typically not as smoothly. It may also increase slightly by the end of the training, which indicates that the model is starting to overfit.

In [ ]:
# Train model with early stopping and class weights; save best model
print('Training model...')
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=16)
model_checkpoint = keras.callbacks.ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)
model.fit(x_train, y_train, epochs=100, batch_size=128, validation_data=(x_val, y_val), class_weight=class_weights,
          callbacks=[early_stopping, model_checkpoint])

# Plot learning curves
plot_learning_curves(model, block=False)

### Plot some filters from the trained model

Among other model parameters, the training has generated some convolution filters. It's instructuve to inspect these filters: Since the windows of class `1` (the ones containing the bell sound) have high magnitudes at certain frequencies, there will typically one or more filters that likewise have high values at these frequencies. You should see the amplified frequencies as yellow horizontal bands in the spectrogram window of a class `1` example, and some yellow cells in the same rows in some of the convolution filters.

In [ ]:
# Plot the first window that has a positive label
plot_first_positive_window(x, y, block=False)

# Plot the 8 convolution filters of the first layer
plot_convolution_filters(8, model.layers[0], block=False)

### Evaluate the model

Inspecting evaluation metrics and plots from the trained model is important for two reasons:

* To see if the model is good enough or if there are any problems.
* To figure out what to do if it's not good enough.

Among evaluation metrics, we will look at the following:

* Validation loss: The value of the loss function computed over the validation set. This is the number to look at to compare different models: the lower loss, the better. Since there is some randomness in the training, simply rerunning the training may produce a better model. Notice that it's not enough to just rerun the training cell in this notebook, because training will just continue from where it was stopped - the model must be rebuilt from scratch, so restart from Build and compile the model.
* Validation accuracy: The classification accuracy computed over the validation set. This number is misleading on unbalanced data sets like ours and should not be used.
* Test loss: The value of the loss function computed over the test set. This number should not be considerably higher than the validation loss. A higher test loss may indicate that the model is overfit.
* Test accuracy: The classification accuracy computed over the test set. Again this is misleading on unbalanced data sets, but with a balanced data set, this number is the most intuitive evaluation metric.

The next four numbers count the number of true and false positive and negative classifications on the test set. The sum of the four numbers is equal to the number of examples in the test set. Other commonly used evaluation metrics like precision and recall can be derived from these numbers.

* True positives: The number of correct `1` classifcations.
* True negatives: The number of correct `0` classifactions.
* False positives: The number of `1` classifications on windows that were labeled `0`, indicating that the classifier detected a bell sound when it wasn't actually there.
* False negatives: The number of `0` classifications on windows that were labeled `1`, indicating that the classifier didn't detect a bell sound when it was actually there.

It's important to note that these numbers count window classifications, which is not the same as bell sound instances, since multiple window classifications happen during a single bell sound instance. Most of the false positives and negatives are typically just that the classifier started detecting or stopped detecting the bell sound slightly too early or too late, but that's not a problem in the actual application. To get a better sense of actually problematic misclassifications, it's better to look at a plot of the predictions versus the labels over time.

In [ ]:
# Load best model
model = keras.models.load_model('model.h5')

# Evaluate model on validation and test sets
val_loss, val_accuracy = model.evaluate(x_val, y_val)
test_loss, test_accuracy = model.evaluate(x_test, y_test)
y_pred = model.predict(x_test)

# Print evaluation metrics
print()
print('Validation loss:     %.4f' % val_loss)
print('Validation accuracy: %.4f' % val_accuracy)
print('Test loss:           %.4f' % test_loss)
print('Test accuracy:       %.4f' % test_accuracy)

# Print confusion matrix
y_pred_bool = np.round(y_pred)
confusion_matrix = np.zeros((2, 2))
for i in range(len(y_pred_bool)):
    confusion_matrix[int(y_test[i]), int(y_pred_bool[i, 0])] += 1
print('True positives:     ', int(confusion_matrix[1, 1]))
print('True negatives:     ', int(confusion_matrix[0, 0]))
print('False positives:    ', int(confusion_matrix[0, 1]))
print('False negatives:    ', int(confusion_matrix[1, 0]))

# Plot predictions vs labels
plot_predictions_vs_labels(y_pred, y_test, block=True)

### Export model to C

Finally, the model is converted to a TensorFlow Lite model and then exported to C code, so it can be loaded in the ESP32 application.

In [ ]:
# Convert to TensorFlow Lite model
print('Converting to TensorFlow Lite model...')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Export TensorFlow Lite model to C source files
print('Exporting TensorFlow Lite model to C source files...')
defines = {"SAMPLE_RATE": SAMPLE_RATE,
           "FRAME_SIZE": FRAME_SIZE,
           "FRAME_STRIDE": FRAME_STRIDE,
           "WINDOW_SIZE": WINDOW_SIZE,
           "SPECTRUM_SIZE": SPECTRUM_SIZE,
           "SPECTRUM_MEAN": SPECTRUM_MEAN,
           "SPECTRUM_STD": SPECTRUM_STD}
write_model_h_file("../ESP-32/main/model.h", defines)
write_model_c_file('../ESP-32/main/model.c', tflite_model)

### Print memory usage

In [ ]:
# Save TensorFlow Lite model and print memory
with open("model.tflite", "wb") as f:
    f.write(tflite_model)
calc_mem("model.tflite")